In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cust_demo = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_01_DEMO.csv')
cust_buy = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_02_PDDE.csv')
aff_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_03_COP_U.csv')
goods_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_04_PD_CLAC.csv')
store_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_05_BR.csv')
Lpay_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_06_LPAY.csv')

C:\Users\legg0\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Dataset

Dataset info 확인

* cust : 고객의 고유 식별번호
* ma_fem_dv : 남성, 여성
* ages : 20대 ~ 70대 이상
* zon_hlv : 지역 대분류 코드
* zom_mcls: 지역 중분류 코드
* rct_no : 장바구니 식별 번호(구매 고유 식별번호)
* chnl_dv : 온라인/오프라인 구분 (1: 오프라인 / 2: 온라인)
* pd_c : 상품코드
* pd_nm: 상품 명
* clac_hlv_nm : 상품 카테고리 대분류명
* clac_mcls_nm : 상품 카테고리 중분류명
* br_c : 구매 점포 구분 코드
* de_dt : YYYY/MM/DD
* de_hr : HH
* buy_am : 구매한 금액
* buy_ct : 구매한 수량
* cop_c : A01-06(유통사), B01(숙박업종), C01-2(엔터테인먼트), D01~2: F&B, E01: 렌탈업종, L00(기타제휴사), L01(비제휴사)

* 유통사 제외 엘페이 Rows 제거
* 새로운 Df 정의 (엘페이)

In [3]:
cop = Lpay_info['cop_c'].tolist()
cust_cop = []

for i in range(len(cop)):
    if cop[i][0] != 'A':
        cust_cop.append(i)

Lpay_info.drop(cust_cop, inplace = True, axis = 0)
Lpay_info.reset_index(drop = True, inplace=True)
Lpay_info

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am
0,M629656521,210803210311226,A03,1,20210803,21,10900
1,M205142844,210803140275112,A02,1,20210803,14,9000
2,M737010483,210803040637594,A06,2,20210803,4,36740
3,M707775545,210803140675502,A06,2,20210803,14,138500
4,M848083175,210803180197432,A01,1,20210803,18,14000
...,...,...,...,...,...,...,...
227992,M470112503,210104210651243,A06,2,20210104,21,50920
227993,M470112503,210125110644545,A06,2,20210125,11,51800
227994,M470112503,210104210652271,A06,2,20210104,21,6440
227995,M748878049,210119200278240,A02,1,20210119,20,17600


* 고객정보와 엘페이 결합

In [4]:
Lpay = pd.merge(Lpay_info, cust_demo, on = 'cust', how = 'left')
Lpay['Lpay_use'] = 1

* 유통사 Lpay 사용: 227997
* 유통사 제외 Rows 모두 제거
* 다시 병합하고, 전처리 하기

In [5]:
Lpay.rename(columns = {'rct_no' : 'Lpay_rct_no'}, inplace = True)

## Preprocessing

* 유통사 Data 중복 제거
* 유통사 + 고객

In [6]:
cust_buy = cust_buy.drop_duplicates()
cust_buy_demo = pd.merge(cust_buy, cust_demo, on = 'cust', how = 'left')

* 4381743 -> 4144389 : 약 20만개 중복 데이터 제거

* 유통사 + 고객 + 상품

In [7]:
cust_goods = pd.merge(cust_buy_demo, goods_info, on = 'pd_c', how = 'left')

* 유통사 + 고객 + 상품 + 점포

In [8]:
df_cust = pd.merge(cust_goods, store_info, on = ['br_c', 'cop_c', 'zon_hlv'], how = 'left')

* 점포 결측치 -> 0 (온라인)

In [9]:
df_cust['br_c'].fillna(0, inplace = True)

* 유통사와 엘페이의 병합
* 엘페이와 중복되는 값 제거필요
* merge -> 엘페이와 유통사의 inner join 없애고, outer로 merge
* 하나의 df 완성

###  병합

#### Inner Merge

In [10]:
df1 = pd.merge(Lpay, df_cust, on = ['cust', 'cop_c', 'chnl_dv', 'de_dt', 'de_hr', 'buy_am', 'ma_fem_dv', 'ages', 'zon_hlv'], how = 'inner')
df1_idx = df1[df1['Lpay_use'] == 1].index
Lpay.drop(df1_idx, inplace = True)
Lpay.reset_index(drop = True, inplace=True)

#### Outer Merge

In [11]:
main_df = pd.merge(df_cust, Lpay, on = ['cust', 'cop_c', 'chnl_dv', 'de_dt', 'de_hr', 'buy_am',
       'ma_fem_dv', 'ages', 'zon_hlv'], how = 'outer')
main_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_mcls,Lpay_rct_no,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN,NaN,NaN
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,기타주방일회용품,주방잡화,주방일회용품,Z13001,NaN,NaN
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN,NaN,NaN
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN,NaN,NaN
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,커피/음료,테넌트/음식점,식당,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4304976,M470112503,NaN,2,A06,NaN,NaN,20210104,21,50920.0,NaN,여성,50대,Z16,NaN,NaN,NaN,NaN,2.101042e+14,1.0
4304977,M470112503,NaN,2,A06,NaN,NaN,20210125,11,51800.0,NaN,여성,50대,Z16,NaN,NaN,NaN,NaN,2.101251e+14,1.0
4304978,M470112503,NaN,2,A06,NaN,NaN,20210104,21,6440.0,NaN,여성,50대,Z16,NaN,NaN,NaN,NaN,2.101042e+14,1.0
4304979,M748878049,NaN,1,A02,NaN,NaN,20210119,20,17600.0,NaN,여성,30대,Z10,NaN,NaN,NaN,NaN,2.101192e+14,1.0


### 전처리

#### Columns Drop

In [12]:
main_df.drop(['Lpay_rct_no', 'pd_nm', 'clac_mcls_nm', 'zon_mcls'], axis = 1, inplace = True)
main_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,NaN
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,NaN
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,NaN
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,NaN
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4304976,M470112503,NaN,2,A06,NaN,NaN,20210104,21,50920.0,NaN,여성,50대,Z16,NaN,1.0
4304977,M470112503,NaN,2,A06,NaN,NaN,20210125,11,51800.0,NaN,여성,50대,Z16,NaN,1.0
4304978,M470112503,NaN,2,A06,NaN,NaN,20210104,21,6440.0,NaN,여성,50대,Z16,NaN,1.0
4304979,M748878049,NaN,1,A02,NaN,NaN,20210119,20,17600.0,NaN,여성,30대,Z10,NaN,1.0


* 결측치 있는 값 행 제거

In [13]:
main_df['Lpay_use'].fillna(0, inplace = True)
main_df['Lpay_use'] = main_df['Lpay_use'].astype(int)

In [14]:
main_df = main_df.dropna(axis = 0)
main_df.reset_index(drop=True, inplace=True)
main_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,0
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,0
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4145048,M816318679,E06052115831,2,A06,0,PD0507,20211231,22,50000.0,1.0,여성,40대,Z17,상품권,0
4145049,M816318679,E06052116037,2,A06,0,PD0507,20211231,22,10000.0,1.0,여성,40대,Z17,상품권,0
4145050,M182645944,E06052117103,2,A06,0,PD0294,20211231,23,289000.0,1.0,남성,20대,Z17,남성의류,0
4145051,M533286446,E06052118403,2,A06,0,PD0507,20211231,23,200000.0,1.0,남성,40대,Z10,상품권,0


In [15]:
main_df.isnull().sum().sort_values(ascending = False)

Lpay_use       0
clac_hlv_nm    0
zon_hlv        0
ages           0
ma_fem_dv      0
buy_ct         0
buy_am         0
de_hr          0
de_dt          0
pd_c           0
br_c           0
cop_c          0
chnl_dv        0
rct_no         0
cust           0
dtype: int64

---------------------------------------------------------------------------------------결측치 제거 완료 ---------------------------------------------------------------------------------

* 영수증 중복 처리 -> O
* 전체 컬럼 전처리 -> 
* 샘플링
* Modeling

-- df -> csv 저장 --

#### 중복처리

In [16]:
main_df = main_df.drop_duplicates(['rct_no'], keep = 'first')
main_df[main_df['Lpay_use'] == 1]
main_df.reset_index(drop = True, inplace = True)

In [17]:
main_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,0
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,0
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266584,M816318679,E06052115831,2,A06,0,PD0507,20211231,22,50000.0,1.0,여성,40대,Z17,상품권,0
1266585,M816318679,E06052116037,2,A06,0,PD0507,20211231,22,10000.0,1.0,여성,40대,Z17,상품권,0
1266586,M182645944,E06052117103,2,A06,0,PD0294,20211231,23,289000.0,1.0,남성,20대,Z17,남성의류,0
1266587,M533286446,E06052118403,2,A06,0,PD0507,20211231,23,200000.0,1.0,남성,40대,Z10,상품권,0


* 300만개 -> 120만개

#### Feature Engineering

##### columns drop

In [18]:
main_df.drop(columns=['cust', 'rct_no'], inplace=True, axis = 1)
main_df

C:\Users\legg0\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,0
1,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,0
2,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
3,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
4,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266584,2,A06,0,PD0507,20211231,22,50000.0,1.0,여성,40대,Z17,상품권,0
1266585,2,A06,0,PD0507,20211231,22,10000.0,1.0,여성,40대,Z17,상품권,0
1266586,2,A06,0,PD0294,20211231,23,289000.0,1.0,남성,20대,Z17,남성의류,0
1266587,2,A06,0,PD0507,20211231,23,200000.0,1.0,남성,40대,Z10,상품권,0


##### Types 변환

In [19]:
main_df['buy_ct'] = main_df['buy_ct'].astype('int64')
main_df['buy_am'] = main_df['buy_am'].astype('int64')
main_df.info()

<ipython-input-19-b833e7a56f3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['buy_ct'] = main_df['buy_ct'].astype('int64')
<ipython-input-19-b833e7a56f3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['buy_am'] = main_df['buy_am'].astype('int64')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266589 entries, 0 to 1266588
Data columns (total 13 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   chnl_dv      1266589 non-null  int64 
 1   cop_c        1266589 non-null  object
 2   br_c         1266589 non-null  object
 3   pd_c         1266589 non-null  object
 4   de_dt        1266589 non-null  int64 
 5   de_hr        1266589 non-null  int64 
 6   buy_am       1266589 non-null  int64 
 7   buy_ct       1266589 non-null  int64 
 8   ma_fem_dv    1266589 non-null  object
 9   ages         1266589 non-null  object
 10  zon_hlv      1266589 non-null  object
 11  clac_hlv_nm  1266589 non-null  object
 12  Lpay_use     1266589 non-null  int32 
dtypes: int32(1), int64(5), object(7)
memory usage: 120.8+ MB


##### Object Columns Preprocess

1. cop_c -> 6개라 Pass
2. br_c -> 6000개 이상 Check
3. pd_c -> 20개 이상 Check
4. ma_fem_dv -> 2개 Pass
5. ages -> Pass
6. zon_hlv -> Pass
7. clac_hlv_nm -> 20개 이상 Check

In [20]:
br_idx = main_df['br_c'].value_counts()[41:].index.tolist()
main_df['br_c'] = main_df['br_c'].replace(br_idx, 999)

<ipython-input-20-21d615f8c403>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['br_c'] = main_df['br_c'].replace(br_idx, 999)


In [21]:
main_df['br_c'].value_counts()

999        694585
0           97690
A010002     50789
A010004     30269
A010001     26343
A010026     21062
A020006     19808
A010034     19709
A010037     18461
A010017     17460
A010045     15165
A010015     12540
A010024     11397
A010053     11256
A010020     10769
A010011      9562
A010062      9548
A020013      9382
A010060      9177
A010063      8829
A010028      8804
A010039      8601
A010008      8451
A010003      8424
A010033      8194
A020010      8151
A010013      7976
A010042      7789
A010032      7757
A020036      7620
A010041      7497
A020008      7207
A010006      7155
A010025      6965
A020096      6868
A020037      6826
A010043      6650
A020007      6636
A010058      6399
A010027      6346
A010023      6255
A020023      6217
Name: br_c, dtype: int64

In [22]:
br_idx = main_df['pd_c'].value_counts()[41:].index.tolist()
main_df['pd_c'] = main_df['pd_c'].replace(br_idx, 9999)

<ipython-input-22-9d3319e136c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['pd_c'] = main_df['pd_c'].replace(br_idx, 9999)


In [23]:
main_df['pd_c'].value_counts()

9999      806583
PD0288     31157
PD0232     21682
PD1689     19482
PD1156     17275
PD1683     16597
PD1484     15443
PD1694     14837
PD0949     13835
PD0480     13764
PD0298     13714
PD1692     12970
PD1181     12814
PD0381     12748
PD0218     11972
PD1255     11286
PD1330     11255
PD1690     10836
PD1173     10675
PD0290     10064
PD1486      9972
PD0409      9852
PD0380      9729
PD1333      9259
PD1152      9174
PD0403      8999
PD1685      8998
PD1699      8858
PD0777      8794
PD1387      8694
PD0413      8419
PD0197      8103
PD0228      7526
PD1688      7404
PD1183      7342
PD1684      7221
PD1331      7175
PD1155      6917
PD0957      6588
PD1154      6352
PD0964      6162
PD1827      6062
Name: pd_c, dtype: int64

In [24]:
br_idx = main_df['clac_hlv_nm'].value_counts()[21:].index.tolist()
main_df['clac_hlv_nm'] = main_df['clac_hlv_nm'].replace(br_idx, 99999)

<ipython-input-24-d2cd53014e9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['clac_hlv_nm'] = main_df['clac_hlv_nm'].replace(br_idx, 99999)


In [25]:
main_df['clac_hlv_nm'].value_counts()

99999       249151
테넌트/음식점     125716
과자           86080
음료           71244
채소           63443
대용식          59314
여성의류         56422
유제품          56410
과일           49706
냉장식품         48429
스포츠패션        45560
패션잡화         42238
조리식품         42065
주류           41095
남성의류         38669
축산물          37768
기타(비상품)      34189
화장품/뷰티케어     26659
담배           26315
냉동식품         24763
조미료          21419
문구/사무용품      19934
Name: clac_hlv_nm, dtype: int64

------------------------------------------------------DF -> CSV-------------------------------------------------------

In [26]:
main_df.to_csv("C:/Users/legg0/Desktop/main_df.csv", header = False, index = False)

### 결측치확인

In [27]:
np.mean(real_df[real_df['buy_ct'] == 1]['buy_am'])

NameError: name 'real_df' is not defined

In [ ]:
real_df['clac_hlv_nm'].value_counts()[:20]

In [ ]:
real_df['clac_hlv_nm'].value_counts()[:20]

In [ ]:
df1['br_c'].value_counts()[:20]

In [ ]:
real_df['pd_c'].value_counts()[:20]

* Lpay결제 여부이니까 영수증 groupby해주면 좋음
* Lpay를 사용하는 데 있어서 제품 구매가 큰 영향을 줄까?
* 개인화(고객)의 영역에 있어서는 개인화 마케팅에 초점 맞춰서 그럴 수 있다 판단함
* 다른 컬럼에 대해서 어떤 컬럼이 큰 영향을 줄까?

In [ ]:
tmp = tmp_df.duplicated(subset = ['rct_no'])
tmp

In [ ]:
main_df['br_c'].value_counts()

In [ ]:
main_df['clac_hlv_nm'].value_counts()

 * 어짜피 Lpay 사용 여부 판단 모델이니 그룹바이해서 한사람이 한 번 결제하는 거니 묶어버림
 * 120만개 target -> 0
 * 6만개 target -> 1
 * 오버 샘플링 

In [ ]:
real_df.columns

* groupby 해주면 120만개로 확줄어드는 것 확인할 수 있음
* target값 Lpay는 중복없음 확인함 20만개 
* 100만: 20만
* 결측치 처리만 어떻게하면 좋은 Model 만들 수 있을 것 같은데
* 제품 3개 중 하나 또는 두개만 남겨도 되지 않을까?
* 데이터를 어느 정도 가져갈 것이며, 결측치는 어떻게 해결해야 할지 감이안옴


* 하나 대안은 언더샘플링 진행하고, 홀드 아웃 통해서 모델 성능 올리는 방안

## Modeling

* Sampling 어떻게 할지 -> 
*

In [ ]:
pip install imbalanced-learn

In [ ]:
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

In [ ]:
X = tmp_df.drop('Lpay_use', axis = 1)
Y = tmp_df['Lpay_use']

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y)

In [ ]:
train_Y.value_counts()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *

KNN_model = KNN(n_neighbors = 11).fit(train_X, train_Y)
pred_Y = KNN_model.predict(test_X)
print(recall_score(test_Y, pred_Y))
print(accuracy_score(test_Y, pred_Y))

In [ ]:
undersample = RandomUnderSampler(sampling_strategy='majority')

In [ ]:
one-hot -> 태준이 모델의 문제점은 클래스 불균형 맞추고 각자 모델 돌립시다
지금 하기로한 데이터
건하씨 csv 